## 0. Kütüphane Tanımlaması

Bu adımda, görsel işleme, veri hazırlama, makine öğrenimi modeli oluşturma ve eğitim için gerekli olan kütüphaneler içe aktarılıyor. OpenCV (cv2) görselleri işlemek, NumPy veri yapıları ve matematiksel işlemler, scikit-learn veri ayırma ve düzenleme, TensorFlow Keras ise derin öğrenme modellerini oluşturmak ve eğitmek için kullanılıyor. Görsellerin yeniden boyutlandırılması, veri artırma (augmentation), sinir ağı katmanlarının oluşturulması ve modelin optimize edilmesi için gerekli tüm araçlar bu kütüphanelerle sağlanıyor.

In [1]:
# Gerekli kütüphaneleri içe aktar
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from sklearn.utils import shuffle

## 1.Veriyi Okuma 

Veri setiyle çalışmak için gerekli dizinler ve parametreler tanımlanmıştır. Görsellerin bulunduğu ana dizin (data_dir) ve işlenmiş, manipüle edilmiş veya beyaz dengesi düzeltilmiş görseller için hedef dizinler belirlenmiştir. Ayrıca, üzerinde çalışılacak 10 hayvan sınıfı seçilmiş ve her sınıf için maksimum 650 görsel işleme alınacak şekilde sınırlandırılmıştır. Görsellerin boyutu 128x128 piksel olarak yeniden boyutlandırılacaktır. Bu ayarlar, veri işleme ve analiz adımlarını düzenlemek için temel yapı taşlarını oluşturur.

In [2]:

# Veri seti dizinleri
data_dir = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"
output_dir = "/kaggle/working/selected_animals_dataset"
processed_dir = "/kaggle/working/processed_dataset"
manipulated_dir = "/kaggle/working/manipulated_images"
wb_dir = "/kaggle/working/wb_corrected_images"

# Seçilen hayvan sınıfları ve parametreler
selected_classes = [
    "collie", "dolphin", "elephant", "fox", "moose", 
    "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"
]
max_images_per_class = 650
image_size = (128, 128)

## 1.Veriyi Okuma ve İşleme

çıktı dizinleri temizlenerek yeniden oluşturulmuş ve seçilen hayvan sınıflarındaki görseller işlenmiştir. Her sınıf için belirlenen maksimum görsel sayısı kadar görsel alınarak, 128x128 piksele yeniden boyutlandırılmış ve normalize edilmiştir (0-1 aralığına getirilmiştir). İşlenmiş görseller, ilgili sınıf isimlerine göre organize edilerek hedef dizine kaydedilmiştir. Bu işlem, veri setini model eğitimi için uygun hale getirmiştir.

In [3]:
# Çıktı dizinlerini temizle ve oluştur
for dir_path in [output_dir, processed_dir, manipulated_dir, wb_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Görselleri seç, yeniden boyutlandır ve normalize et
for animal_class in selected_classes:
    class_path = os.path.join(data_dir, animal_class)
    if os.path.exists(class_path):
        images = sorted(os.listdir(class_path))[:max_images_per_class]
        processed_class_path = os.path.join(processed_dir, animal_class)
        os.makedirs(processed_class_path, exist_ok=True)

        for image_name in images:
            img_path = os.path.join(class_path, image_name)
            img = cv2.imread(img_path)
            if img is not None:
                img_resized = cv2.resize(img, image_size)
                img_normalized = img_resized / 255.0
                processed_image_path = os.path.join(processed_class_path, image_name)
                cv2.imwrite(processed_image_path, (img_normalized * 255).astype(np.uint8))

print(f"Veri seti '{processed_dir}' dizininde işlenmiştir.")


Veri seti '/kaggle/working/processed_dataset' dizininde işlenmiştir.


## 2. Görüntü Manipülasyonu ve Beyaz Dengesi Düzenleme Adımı







işlenmiş görseller üzerinde manipülasyon ve beyaz dengesi düzenleme işlemleri yapılmıştır. Manipülasyon fonksiyonu, kontrast ve parlaklığı artırarak görselleri manipüle edip belirlenen dizine kaydetmiştir. Ardından, beyaz dengesi fonksiyonu her bir görselin renk kanallarını ortalama gri seviyesine göre ayarlayarak beyaz dengesini düzenlemiş ve sonuçları ayrı bir dizine kaydetmiştir. Bu işlemler, görsellerin kalitesini artırarak model eğitimi için daha uygun hale getirilmesini sağlamıştır.

In [4]:
# Manipülasyon Fonksiyonu
def get_manipulated_images(input_dir, output_dir, brightness=1.5, contrast=1.2):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                manipulated_img = cv2.convertScaleAbs(img, alpha=contrast, beta=brightness * 50)
                save_path = os.path.join(output_class_path, img_name)
                cv2.imwrite(save_path, manipulated_img)

get_manipulated_images(processed_dir, manipulated_dir)
print(f"Manipüle edilmiş görüntüler '{manipulated_dir}' dizinine kaydedildi.")

# Beyaz Dengesi Fonksiyonu
def get_wb_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                avg_b, avg_g, avg_r = np.mean(img, axis=(0, 1))
                avg_gray = (avg_b + avg_g + avg_r) / 3
                img[:, :, 0] = np.clip(img[:, :, 0] * (avg_gray / avg_b), 0, 255)
                img[:, :, 1] = np.clip(img[:, :, 1] * (avg_gray / avg_g), 0, 255)
                img[:, :, 2] = np.clip(img[:, :, 2] * (avg_gray / avg_r), 0, 255)
                save_path = os.path.join(output_class_path, img_name)
                cv2.imwrite(save_path, img.astype(np.uint8))

get_wb_images(manipulated_dir, wb_dir)
print(f"Beyaz dengesi sağlanmış görüntüler '{wb_dir}' dizinine kaydedildi.")


Manipüle edilmiş görüntüler '/kaggle/working/manipulated_images' dizinine kaydedildi.
Beyaz dengesi sağlanmış görüntüler '/kaggle/working/wb_corrected_images' dizinine kaydedildi.


## 3. Görsellerin Yüklenmesi ve Etiketlenmesi

 işlenmiş, manipüle edilmiş ve beyaz dengesi sağlanmış görseller belirlenen dizinlerden yüklenmiş ve sınıflarına göre etiketlenmiştir. Her görsel 0-1 aralığında normalize edilerek bir NumPy dizisine eklenmiş, etiketler ise sınıf adlarına karşılık gelen sayısal değerlerle eşleştirilmiştir. Bu işlem, model eğitimi için görsellerin ve etiketlerin organize bir şekilde hazırlanmasını sağlamıştır.

In [5]:
def load_images_from_directory(input_dir, class_names):
    X, y = [], []
    class_mapping = {cls: idx for idx, cls in enumerate(class_names)}

    for class_name in class_names:
        class_path = os.path.join(input_dir, class_name)
        if os.path.exists(class_path):
            images = sorted(os.listdir(class_path))
            for img_name in images:
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    X.append(img / 255.0)
                    y.append(class_mapping[class_name])

    return np.array(X), np.array(y)

# Yükleme işlemi
X_train, y_train = load_images_from_directory(processed_dir, selected_classes)
X_manipulated_train, y_manipulated_train = load_images_from_directory(manipulated_dir, selected_classes)
X_wb_train, y_wb_train = load_images_from_directory(wb_dir, selected_classes)

print(f"Manipüle Edilmiş: {X_manipulated_train.shape}, Beyaz Dengeli: {X_wb_train.shape}")


Manipüle Edilmiş: (6500, 128, 128, 3), Beyaz Dengeli: (6500, 128, 128, 3)


## 4. Veri Birleştirme ve CNN Model Eğitimi

işlenmiş, manipüle edilmiş ve beyaz dengesi sağlanmış tüm görseller birleştirilmiş ve etiketler one-hot encoding formatına dönüştürülerek model eğitimi için hazırlanmıştır. Ardından, bir Convolutional Neural Network (CNN) modeli tanımlanmış ve optimize edilmiştir. Model, görsellerdeki özellikleri öğrenmek için evrişim, havuzlama, normalizasyon ve yoğun katmanlar içermektedir. Model, tümleştirilmiş veri üzerinde 20 dönem boyunca eğitilmiştir. Bu işlem, sınıflandırma performansını artırmak için farklı veri varyasyonlarından faydalanmayı amaçlamaktadır.

In [6]:
# Veri birleştirme
X_combined_train = np.concatenate([X_train, X_manipulated_train, X_wb_train], axis=0)
y_combined_train = np.concatenate([y_train, y_manipulated_train, y_wb_train], axis=0)

# Etiketleri one-hot encoding
y_combined_train_one_hot = to_categorical(y_combined_train, num_classes=len(selected_classes))

# CNN Model
model = Sequential([
    Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, kernel_regularizer=l2(0.02)),
    LeakyReLU(alpha=0.1),
    Dropout(0.7),
    Dense(len(selected_classes), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Eğitim
history = model.fit(X_combined_train, y_combined_train_one_hot, epochs=20, batch_size=32)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.2873 - loss: 7.2661
Epoch 2/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.4813 - loss: 5.1311
Epoch 3/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.5912 - loss: 3.8039
Epoch 4/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.6732 - loss: 2.9027
Epoch 5/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7327 - loss: 2.2810
Epoch 6/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.7942 - loss: 1.8403
Epoch 7/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.8364 - loss: 1.5527
Epoch 8/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.8743 - loss: 1.3082
Epoch 9/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.8819 - loss: 1.2118
Epoch 10/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.8976 - loss: 1.1089
Epoch 11/20
610/610 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.9069 - loss: 1.0582
Epoch 12/20
610/610 ━━━━━━━━━━

## 4. Test Veri Değerlendirmesi ve Performans Karşılaştırması


Modelin performansını değerlendirmek için işlenmiş test veri seti yüklenmiş ve one-hot encoding ile etiketlenmiştir. Model, orijinal test verileri üzerinde değerlendirilerek kayıp ve doğruluk değerleri hesaplanmıştır. Ardından, test veri seti manipüle edilerek parlaklık artırılmış, döndürme işlemi uygulanmış ve yeniden ölçeklendirilmiştir. Manipüle edilmiş test veri seti üzerinde model yeniden değerlendirilerek doğruluk ve kayıp sonuçları raporlanmıştır.

Son olarak, manipüle edilmiş veri setine beyaz dengesi işlemi uygulanmış ve model üzerinde aynı değerlendirme süreci tekrarlanmıştır. Bu işlemler, modelin veri manipülasyonlarına ve beyaz dengesi düzenlemelerine karşı performans değişimini ölçmeyi amaçlamaktadır. Elde edilen sonuçlar karşılaştırılarak modelin farklı veri varyasyonlarına ne kadar duyarlı olduğu analiz edilmiştir.

In [7]:
# Test verilerini yükleme
X_test, y_test = load_images_from_directory(processed_dir, selected_classes)
y_test_one_hot = to_categorical(y_test, num_classes=len(selected_classes))

# Model değerlendirme
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot, verbose=2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


204/204 - 3s - 14ms/step - accuracy: 0.9957 - loss: 0.6951
Test Loss: 0.695090651512146, Test Accuracy: 0.9956923127174377


In [8]:
# 4. Manipüle Edilmiş Test Seti

def manipulate_images(X):
    manipulated_images = []
    for img in X:
        img_8bit = (img * 255).astype(np.uint8)
        manipulated = cv2.convertScaleAbs(img_8bit, alpha=2.0, beta=0)
        manipulated = cv2.rotate(manipulated, cv2.ROTATE_90_CLOCKWISE)
        manipulated_images.append(manipulated / 255.0)
    return np.array(manipulated_images, dtype=np.float32)

X_test_manipulated = manipulate_images(X_test)
manipulated_loss, manipulated_accuracy = model.evaluate(X_test_manipulated, y_test_one_hot, verbose=1)
print(f"Manipüle Edilmiş Test Doğruluğu: {manipulated_accuracy * 100:.2f}%")

# 5. Beyaz Dengesi

def apply_white_balance(X):
    wb_images = []
    for img in X:
        avg_b, avg_g, avg_r = cv2.mean(img)[:3]
        avg_gray = (avg_b + avg_g + avg_r) / 3
        img[:, :, 0] = np.clip(img[:, :, 0] * (avg_gray / avg_b), 0, 255)
        img[:, :, 1] = np.clip(img[:, :, 1] * (avg_gray / avg_g), 0, 255)
        img[:, :, 2] = np.clip(img[:, :, 2] * (avg_gray / avg_r), 0, 255)
        wb_images.append(img / 255.0)
    return np.array(wb_images, dtype=np.float32)

X_test_wb = apply_white_balance(X_test_manipulated)
wb_loss, wb_accuracy = model.evaluate(X_test_wb, y_test_one_hot, verbose=1)
print(f"Beyaz Dengesi Test Doğruluğu: {wb_accuracy * 100:.2f}%")

# 6. Karşılaştırma ve Raporlama
print("Sonuçlar:")
original_loss, original_accuracy = model.evaluate(X_test, y_test_one_hot, verbose=1)
print(f"Orijinal Test Seti Doğruluğu: {original_accuracy * 100:.2f}%")
print(f"Manipüle Test Seti Doğruluğu: {manipulated_accuracy * 100:.2f}%")
print(f"Beyaz Dengesi Uygulanmış Test Seti Doğruluğu: {wb_accuracy * 100:.2f}%")

204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3905 - loss: 3.7628
Manipüle Edilmiş Test Doğruluğu: 36.17%
204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0173 - loss: 4.1850
Beyaz Dengesi Test Doğruluğu: 10.00%
Sonuçlar:
204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9889 - loss: 0.7156
Orijinal Test Seti Doğruluğu: 99.57%
Manipüle Test Seti Doğruluğu: 36.17%
Beyaz Dengesi Uygulanmış Test Seti Doğruluğu: 10.00%
